# 4.4 隐马尔可夫模型
## 4.4.1 马尔可夫链
指时间和状态都是离散的马尔可夫过程，该过程具有如下特性：在已知系统当前的状态条件下，他未来的演变不依赖过去的演变。即系统在状态转移的过程中，第T+1次结果只受第T次结果影响，只与当前状态有关，与过去状态（初始状态和此前转移的所有状态）无关。

## 4.4.2 隐马尔可夫模型
一个隐马尔可夫模型由两组状态集合和三组概率集合组成：
- 隐藏状态：系统中隐含的，但真实存在的状态序列，如：一个句子中每个词的词性；
- 状态转移矩阵：隐状态空间，一个隐状态转移到另一个状态的概率分布矩阵，如一个词性转移到另一个词性；
- 观测状态：显示的状态序列，如句子中的每个字符或单词；
- 初始概率（$\pi$向量）：初始时刻观测状态到隐藏状态的矩阵，如每个句子的开头词语的词性；
-  发射概率矩阵：观测状态到隐藏状态的概率矩阵。

对于HMM来说，有如下三个重要假设：
- 假设1:马尔可夫假设（状态构成一阶马尔库夫链）
- 假设2:不动性假设（状态与具体时间无关）
- 假设3:输出独立性假设（输出仅与当前状态有关）

但在自然语言处理的实际情况中，每个字符都不仅仅与当前状态有关，即不符合输出独立性假设和不动性假设，这也造就了HMM算法自然语言处理中的局限性

## 4.4.3 HMM实例（P172）
- 隐藏状态集合S：{晴天，阴天，雨天}
- 状态转移矩阵
- viterbi 算法，删掉最不可能的路径

In [2]:
# 实例实现如下
import numpy as np
from hmmlearn import hmm

In [3]:
# 暴力法
S = ('晴天', '阴天', '雨天')     # 隐藏状态集合分别代表晴天、阴天、雨天
O = (0, 1, 3)   # 观测序列集合，分别代表干旱、干燥、湿润、潮湿
Pi = np.array([0.63, 0.17, 0.20])     # 初始状态
A = np.array([
    [0.5, 0.375, 0.125],
    [0.25, 0.125, 0.625],
    [0.25, 0.375, 0.375]
])   # 状态转移矩阵
B = np.array([
    [0.6, 0.20, 0.15, 0.05],
    [0.25, 0.25, 0.25, 0.25],
    [0.05, 0.10, 0.35, 0.50]
])  # 发射矩阵

# 实际观测序列：干旱-干燥-潮湿
# 第一天的天气状况：与所有天气的初始状态有关与当天的环境适度有关
state1 = np.multiply(Pi, B[:, 0].T)
print('第一天干旱时，各种天气的概率', state1)
print('第一天干旱时，最有可能的天气', state1.argmax())

# 第二天的天气状况与天气的转移情况有关，与当天的环境湿度有关
# 仍为晴天的概率：(晴天-晴天+阴天-晴天+雨天-晴天) * 干燥
state2 = np.multiply(np.dot(state1, A).T, B[:, 1])
print('第二天干燥时，各种天气的概率', state2.T)
print('第二天干燥时，最有可能的天气', state2.argmax())

# 第三天的天气状况与前一天的天气和当天的湿度有关
state3 = np.multiply(np.dot(state2.T, A).T, B[:, 3])
print('第三天潮湿时，各种天气的概率', state3.T)
print('第三天潮湿时，最有可能的天气', state3.argmax())

# 所以这三天的最可能的天气为晴天-晴天-雨天

第一天干旱时，各种天气的概率 [0.378  0.0425 0.01  ]
第一天干旱时，最有可能的天气 0
第二天干燥时，各种天气的概率 [0.040425   0.03770312 0.00775625]
第二天干燥时，最有可能的天气 0
第三天潮湿时，各种天气的概率 [0.00157887 0.00569521 0.01576309]
第三天潮湿时，最有可能的天气 2


## 4.4.4 viterbi 算法

In [9]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    """
    obs: 观测序列
    states: 隐藏状态
    start_p: 初始状态分布矩阵
    trans_p: 状态转移举着
    emit_p: 发射矩阵，由状态表现为观测序列的概率矩阵
    """
    # 计算初始状态
    # 存储各时刻的状态可能值 V[时间][隐状态] = 概率
    V = [{}]
    # 初始状态时的各隐状态的概率值
    for state in states:
        V[0][state] = start_p[state] * emit_p[state][obs[0]]
    # 此处计算无误，在寻找最优路径时有误
    for t in range(1, len(obs)):
        V.append({})
        for state in states:
            # t 时刻各隐状态的概率 = t-1时刻状的状态last_state的概率 * last_state
            # 转移到state的概率 * state状态下的观测值概率
            V[t][state] = max([(V[t - 1][ls] * trans_p[ls][state] * emit_p[state][obs[t]])
                               for ls in states])
    
    # 寻找最优路径错误
    print(V)
    result = [max(vec.items(), key=lambda x: x[1]) for vec in V]
#     for vector in V:
#         temp = max(vector.items(), key=lambda x: x[1])
#         result.append(temp)
    return result

In [5]:
states = ('晴天', '阴天', '雨天')
obs = ('干旱', '干燥', '潮湿')
start_p = {'晴天': 0.63, '阴天': 0.17, '雨天': 0.20}
trans_p = {
    '晴天': {'晴天': 0.5, '阴天': 0.375, '雨天': 0.125},
    '阴天': {'晴天': 0.25, '阴天': 0.125, '雨天': 0.625},
    '雨天': {'晴天': 0.25, '阴天': 0.375, '雨天': 0.375},
}
# 在当前隐状态下的观测值的分布概率
emit_p = {
    '晴天': {'干旱': 0.60, '干燥': 0.20, '湿润': 0.15, '潮湿': 0.05},
    '阴天': {'干旱': 0.25, '干燥': 0.25, '湿润': 0.25, '潮湿': 0.25},
    '雨天': {'干旱': 0.05, '干燥': 0.10, '湿润': 0.35, '潮湿': 0.50}
}

In [10]:
viterbi(obs, states, start_p, trans_p, emit_p)

[{'晴天': 0.378, '阴天': 0.0425, '雨天': 0.010000000000000002}, {'晴天': 0.0378, '阴天': 0.0354375, '雨天': 0.004725}, {'晴天': 0.0009450000000000001, '阴天': 0.00354375, '雨天': 0.01107421875}]


[('晴天', 0.378), ('晴天', 0.0378), ('雨天', 0.01107421875)]

In [119]:
# 使用hmmlearn
model = hmm.MultinomialHMM(n_components=len(states))   # 状态数量

model.startprob_ = Pi    # 初始化概率矩阵
model.transmat_ = A
model.emissionprob_ = B

In [123]:
O = (2, 1, 2)
model.decode(np.array(O).reshape(-1, 1), algorithm='viterbi')
t = model.predict(np.array(O).reshape(-1, 1))
print(list(map(lambda x: S[x], t)))
model.predict_proba(np.array(O).reshape(-1, 1))

['晴天', '阴天', '雨天']


array([[0.50077599, 0.15754917, 0.34167484],
       [0.32972547, 0.49566574, 0.17460879],
       [0.20596986, 0.26170411, 0.53232603]])

In [15]:
# 正确的
def viterbit_dict(obs, states, pi_pro, A_pro, B_pro):
    # init path: path[s] represents the path ends with s
    path_v = [{}]
    # 存储每个时刻的最优路径
    path_max = {}

    # 计算初始时的状态概率
    for state in states:
        path_v[0][state] = pi_pro[state] * B_pro[state].get(obs[0], 0)
        path_max[state] = [state]

    print(path_max)
    # print(path_v)
    # 计算第一个观测值之后的观测值
    for o in range(1, len(obs)):
        path_v.append({})
        new_path = {}

        # 每个状态到每个观测值的发射概率
        for state in states:
            # 最大概率和其对应的状态
            temp_pro, temp_state = max(((path_v[o - 1][l] * A_pro[l][state] * B_pro[state].get(obs[o], 0), l) for l in states))
            path_v[o][state] = temp_pro

            new_path[state] = path_max[temp_state] + [state]
        path_max = new_path
        # print(path_v)
        print(path_max)
    best_path_pro, last_state = max((
        path_v[len(obs) - 1][s], s) for s in states)
    print(path_v)
    print(last_state)
    print('最可能的隐状态序列', path_max[last_state])
    print(best_path_pro)

In [14]:
viterbit_dict(obs, states, start_p, trans_p, emit_p)

{'晴天': ['晴天'], '阴天': ['阴天'], '雨天': ['雨天']}
{'晴天': ['晴天', '晴天'], '阴天': ['晴天', '阴天'], '雨天': ['晴天', '雨天']}
{'晴天': ['晴天', '晴天', '晴天'], '阴天': ['晴天', '晴天', '阴天'], '雨天': ['晴天', '阴天', '雨天']}
[{'晴天': 0.378, '阴天': 0.0425, '雨天': 0.010000000000000002}, {'晴天': 0.0378, '阴天': 0.0354375, '雨天': 0.004725}, {'晴天': 0.0009450000000000001, '阴天': 0.00354375, '雨天': 0.01107421875}]
雨天
['晴天', '阴天', '雨天']
0.01107421875


# 4.5 结构化平均感知机模型

## 4.5.1 原理
每个字都有一个标注，每个字都属于一个类别，即转化为分类问题。共有 BMES 四个类别

# 4.6 条件随机场
## 4.6.1 随机场
随机场包含两个要素：1. 样本空间集合；2. 单个样本空间中的随机变量集合。

已有的随机场：吉布斯随机场，马尔可夫随机场，条件随机场，高斯随机场。求解一个随机场，就是要找到有多少种不同的样本空间。

## 4.6.2 无向图的团与因子分解
无向图G的某个子图S，若S中任何两个节点均有边，则S称为G的团。若C是G的一个团，并且不能再加入任何一个G的节点，使其成为团，则称C是G的最大团。

对于一个无向图而言，其联合分布概率可以表示成最大团上的**随机变量函数**的乘积的形式,这个操作成为无向图的因子分解。

计算无向图的联合分布概率公式如下，称为Hammersley-Clifford定理。
$$P(Y)=\frac{1}{Z}\prod_C \psi_C(Y_C)$$
$$Z = \sum_Y \prod_C \psi_C(Y_C)$$
式中：

* $C$ -- 无向图最大团；
* $Y_C$ -- 最大团C中的样本集合；

## 4.6.3 线性链条件随机场
